In [122]:
!pip install transformers

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


In [123]:
!pip install datasets

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


In [124]:
import transformers
from transformers import BertModel, AutoTokenizer, BertTokenizer, PreTrainedTokenizerFast, AdamW, get_linear_schedule_with_warmup
from transformers import DistilBertForSequenceClassification, Trainer, TrainingArguments
from transformers import DistilBertTokenizerFast
import torch.nn.functional as F
from datasets import load_dataset

import torch
import numpy as np
import pandas as pd
import seaborn as sns
from pylab import rcParams
import matplotlib.pyplot as plt
from matplotlib import rc
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix, classification_report
from sklearn.metrics import precision_recall_fscore_support, accuracy_score
from collections import defaultdict
from textwrap import wrap
from torch import nn, optim
from torch.nn.utils import clip_grad_norm_
from torch.utils.data import Dataset, DataLoader
from torch.nn import CrossEntropyLoss

import random

In [125]:
PRE_TRAINED_MODEL_NAME = 'bert-base-uncased'

In [126]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# Data

In [127]:
imdb_dataset = load_dataset('imdb')

  0%|          | 0/3 [00:00<?, ?it/s]

In [128]:
imdb_dataset = imdb_dataset.shuffle(seed=42)

In [129]:
train_texts = imdb_dataset["train"]["text"][:4000]
train_labels = imdb_dataset["train"]["label"][:4000]
test_texts = imdb_dataset["test"]["text"][:1000]
test_labels = imdb_dataset["test"]["label"][:1000]

In [130]:
labels_num = len(set(test_labels))
labels_num

2

In [131]:
del imdb_dataset

In [132]:
from sklearn.model_selection import train_test_split
train_texts, val_texts, train_labels, val_labels = train_test_split(train_texts, train_labels, test_size=.2)

In [133]:
tokenizer = DistilBertTokenizerFast.from_pretrained(PRE_TRAINED_MODEL_NAME)

https://huggingface.co/bert-base-uncased/resolve/main/tokenizer_config.json not found in cache or force_download set to True, downloading to /root/.cache/huggingface/transformers/tmpsvpmg9hx


Downloading:   0%|          | 0.00/28.0 [00:00<?, ?B/s]

storing https://huggingface.co/bert-base-uncased/resolve/main/tokenizer_config.json in cache at /root/.cache/huggingface/transformers/c1d7f0a763fb63861cc08553866f1fc3e5a6f4f07621be277452d26d71303b7e.20430bd8e10ef77a7d2977accefe796051e01bc2fc4aa146bc862997a1a15e79
creating metadata file for /root/.cache/huggingface/transformers/c1d7f0a763fb63861cc08553866f1fc3e5a6f4f07621be277452d26d71303b7e.20430bd8e10ef77a7d2977accefe796051e01bc2fc4aa146bc862997a1a15e79
https://huggingface.co/bert-base-uncased/resolve/main/vocab.txt not found in cache or force_download set to True, downloading to /root/.cache/huggingface/transformers/tmpxp__1ecs


Downloading:   0%|          | 0.00/226k [00:00<?, ?B/s]

storing https://huggingface.co/bert-base-uncased/resolve/main/vocab.txt in cache at /root/.cache/huggingface/transformers/45c3f7a79a80e1cf0a489e5c62b43f173c15db47864303a55d623bb3c96f72a5.d789d64ebfe299b0e416afc4a169632f903f693095b4629a7ea271d5a0cf2c99
creating metadata file for /root/.cache/huggingface/transformers/45c3f7a79a80e1cf0a489e5c62b43f173c15db47864303a55d623bb3c96f72a5.d789d64ebfe299b0e416afc4a169632f903f693095b4629a7ea271d5a0cf2c99
https://huggingface.co/bert-base-uncased/resolve/main/tokenizer.json not found in cache or force_download set to True, downloading to /root/.cache/huggingface/transformers/tmppewfh_jr


Downloading:   0%|          | 0.00/455k [00:00<?, ?B/s]

storing https://huggingface.co/bert-base-uncased/resolve/main/tokenizer.json in cache at /root/.cache/huggingface/transformers/534479488c54aeaf9c3406f647aa2ec13648c06771ffe269edabebd4c412da1d.7f2721073f19841be16f41b0a70b600ca6b880c8f3df6f3535cbc704371bdfa4
creating metadata file for /root/.cache/huggingface/transformers/534479488c54aeaf9c3406f647aa2ec13648c06771ffe269edabebd4c412da1d.7f2721073f19841be16f41b0a70b600ca6b880c8f3df6f3535cbc704371bdfa4
loading file https://huggingface.co/bert-base-uncased/resolve/main/vocab.txt from cache at /root/.cache/huggingface/transformers/45c3f7a79a80e1cf0a489e5c62b43f173c15db47864303a55d623bb3c96f72a5.d789d64ebfe299b0e416afc4a169632f903f693095b4629a7ea271d5a0cf2c99
loading file https://huggingface.co/bert-base-uncased/resolve/main/tokenizer.json from cache at /root/.cache/huggingface/transformers/534479488c54aeaf9c3406f647aa2ec13648c06771ffe269edabebd4c412da1d.7f2721073f19841be16f41b0a70b600ca6b880c8f3df6f3535cbc704371bdfa4
loading file https://hugg

Downloading:   0%|          | 0.00/570 [00:00<?, ?B/s]

storing https://huggingface.co/bert-base-uncased/resolve/main/config.json in cache at /root/.cache/huggingface/transformers/3c61d016573b14f7f008c02c4e51a366c67ab274726fe2910691e2a761acf43e.37395cee442ab11005bcd270f3c34464dc1704b715b5d7d52b1a461abe3b9e4e
creating metadata file for /root/.cache/huggingface/transformers/3c61d016573b14f7f008c02c4e51a366c67ab274726fe2910691e2a761acf43e.37395cee442ab11005bcd270f3c34464dc1704b715b5d7d52b1a461abe3b9e4e
loading configuration file https://huggingface.co/bert-base-uncased/resolve/main/config.json from cache at /root/.cache/huggingface/transformers/3c61d016573b14f7f008c02c4e51a366c67ab274726fe2910691e2a761acf43e.37395cee442ab11005bcd270f3c34464dc1704b715b5d7d52b1a461abe3b9e4e
Model config BertConfig {
  "_name_or_path": "bert-base-uncased",
  "architectures": [
    "BertForMaskedLM"
  ],
  "attention_probs_dropout_prob": 0.1,
  "classifier_dropout": null,
  "gradient_checkpointing": false,
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.1,
  "h

In [134]:
train_encodings = tokenizer(train_texts, truncation=True, padding=True, return_token_type_ids=False, return_attention_mask=True, max_length=512)
val_encodings = tokenizer(val_texts,truncation=True, padding=True, return_token_type_ids=False, return_attention_mask=True, max_length=512)
test_encodings = tokenizer(test_texts, truncation=True, padding=True, return_token_type_ids=False, return_attention_mask=True, max_length=512)

In [135]:
class IMDbDataset(torch.utils.data.Dataset):
    def __init__(self, encodings, labels):
        self.encodings = encodings
        self.labels = labels

    def __getitem__(self, idx):
        item = {key: torch.tensor(val[idx]) for key, val in self.encodings.items()}
        item['labels'] = torch.tensor(self.labels[idx])
        return item

    def __len__(self):
        return len(self.labels)

train_dataset = IMDbDataset(train_encodings, train_labels)
val_dataset = IMDbDataset(val_encodings, val_labels)
test_dataset = IMDbDataset(test_encodings, test_labels)

In [136]:
def compute_metrics(pred):
    labels = pred.label_ids
    preds = pred.predictions.argmax(-1)
    precision, recall, f1, _ = precision_recall_fscore_support(labels, preds, average='binary')
    acc = accuracy_score(labels, preds)
    return {
        'accuracy': acc,
        'f1': f1,
        'precision': precision,
        'recall': recall
    }

In [137]:
import os
os.environ["WANDB_DISABLED"] = "true"

In [138]:
training_args = TrainingArguments(
    output_dir='./results',          # output directory
    num_train_epochs=2,              # total number of training epochs
    per_device_train_batch_size=8,  # batch size per device during training
    per_device_eval_batch_size=16,   # batch size for evaluation
    warmup_steps=500,                # number of warmup steps for learning rate scheduler
    weight_decay=0.01,               # strength of weight decay
    logging_dir='./logs',            # directory for storing logs
    logging_steps=10,
    report_to=None,
)

PyTorch: setting up devices
The default value for the training argument `--report_to` will change in v5 (from all installed integrations to none). In v5, you will need to use `--report_to all` to get the same behavior as now. You should start updating your code and make this info disappear :-).
Using the `WAND_DISABLED` environment variable is deprecated and will be removed in v5. Use the --report_to flag to control the integrations used for logging result (for instance --report_to none).


# Train

In [139]:
bert_model = BertModel.from_pretrained(PRE_TRAINED_MODEL_NAME)

loading configuration file https://huggingface.co/bert-base-uncased/resolve/main/config.json from cache at /root/.cache/huggingface/transformers/3c61d016573b14f7f008c02c4e51a366c67ab274726fe2910691e2a761acf43e.37395cee442ab11005bcd270f3c34464dc1704b715b5d7d52b1a461abe3b9e4e
Model config BertConfig {
  "architectures": [
    "BertForMaskedLM"
  ],
  "attention_probs_dropout_prob": 0.1,
  "classifier_dropout": null,
  "gradient_checkpointing": false,
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.1,
  "hidden_size": 768,
  "initializer_range": 0.02,
  "intermediate_size": 3072,
  "layer_norm_eps": 1e-12,
  "max_position_embeddings": 512,
  "model_type": "bert",
  "num_attention_heads": 12,
  "num_hidden_layers": 12,
  "pad_token_id": 0,
  "position_embedding_type": "absolute",
  "transformers_version": "4.16.2",
  "type_vocab_size": 2,
  "use_cache": true,
  "vocab_size": 30522
}

https://huggingface.co/bert-base-uncased/resolve/main/pytorch_model.bin not found in cache or force_down

Downloading:   0%|          | 0.00/420M [00:00<?, ?B/s]

storing https://huggingface.co/bert-base-uncased/resolve/main/pytorch_model.bin in cache at /root/.cache/huggingface/transformers/a8041bf617d7f94ea26d15e218abd04afc2004805632abc0ed2066aa16d50d04.faf6ea826ae9c5867d12b22257f9877e6b8367890837bd60f7c54a29633f7f2f
creating metadata file for /root/.cache/huggingface/transformers/a8041bf617d7f94ea26d15e218abd04afc2004805632abc0ed2066aa16d50d04.faf6ea826ae9c5867d12b22257f9877e6b8367890837bd60f7c54a29633f7f2f
loading weights file https://huggingface.co/bert-base-uncased/resolve/main/pytorch_model.bin from cache at /root/.cache/huggingface/transformers/a8041bf617d7f94ea26d15e218abd04afc2004805632abc0ed2066aa16d50d04.faf6ea826ae9c5867d12b22257f9877e6b8367890837bd60f7c54a29633f7f2f
Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertModel: ['cls.predictions.decoder.weight', 'cls.predictions.bias', 'cls.seq_relationship.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.transform.dense.we

In [140]:
# BertForSequenceClassification source code

class SentimentClassifier(nn.Module):

    def __init__(self, n_classes):

    super().__init__()
    self.bert = BertModel.from_pretrained(PRE_TRAINED_MODEL_NAME)
    self.drop = nn.Dropout(p=0.3)
    self.out = nn.Linear(self.bert.config.hidden_size, n_classes)
    self.n_classes = n_classes

    def forward(self,        
        input_ids=None,
        attention_mask=None,
        token_type_ids=None,
        position_ids=None,
        head_mask=None,
        inputs_embeds=None,
        labels=None,
        output_attentions=None,
        output_hidden_states=None
    ):

    last_hidden_state, pooled_output = self.bert(input_ids=input_ids,
                        attention_mask=attention_mask,
                        return_dict=False)

    output = self.drop(pooled_output)
    logits = self.out(output)

    loss = None
    if labels is not None:
        loss_fct = CrossEntropyLoss()
        loss = loss_fct(logits.view(-1, self.n_classes), labels.view(-1))
    output = (logits,)
    return ((loss,) + output) if loss is not None else output

In [141]:
model = SentimentClassifier(labels_num)
model = model.to(device)

loading configuration file https://huggingface.co/bert-base-uncased/resolve/main/config.json from cache at /root/.cache/huggingface/transformers/3c61d016573b14f7f008c02c4e51a366c67ab274726fe2910691e2a761acf43e.37395cee442ab11005bcd270f3c34464dc1704b715b5d7d52b1a461abe3b9e4e
Model config BertConfig {
  "architectures": [
    "BertForMaskedLM"
  ],
  "attention_probs_dropout_prob": 0.1,
  "classifier_dropout": null,
  "gradient_checkpointing": false,
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.1,
  "hidden_size": 768,
  "initializer_range": 0.02,
  "intermediate_size": 3072,
  "layer_norm_eps": 1e-12,
  "max_position_embeddings": 512,
  "model_type": "bert",
  "num_attention_heads": 12,
  "num_hidden_layers": 12,
  "pad_token_id": 0,
  "position_embedding_type": "absolute",
  "transformers_version": "4.16.2",
  "type_vocab_size": 2,
  "use_cache": true,
  "vocab_size": 30522
}

loading weights file https://huggingface.co/bert-base-uncased/resolve/main/pytorch_model.bin from cache 

In [142]:
model

SentimentClassifier(
  (bert): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(30522, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0): BertLayer(
          (attention): BertAttention(
            (self): BertSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affin

In [143]:
trainer = Trainer(
          model=model,
          args=training_args,
          train_dataset=train_dataset,
          eval_dataset=val_dataset,
          compute_metrics=compute_metrics,
)

In [144]:
trainer.train()

/opt/conda/lib/python3.7/site-packages/transformers/optimization.py:309: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use thePyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  FutureWarning,
***** Running training *****
  Num examples = 3200
  Num Epochs = 2
  Instantaneous batch size per device = 8
  Total train batch size (w. parallel, distributed & accumulation) = 8
  Gradient Accumulation steps = 1
  Total optimization steps = 800


Step,Training Loss
10,0.706700
20,0.694900
30,0.736500
40,0.696000
50,0.709100
60,0.703600
70,0.687500
80,0.687800
90,0.655500
100,0.681100


Saving model checkpoint to ./results/checkpoint-500
Trainer.model is not a `PreTrainedModel`, only saving its state dict.


Training completed. Do not forget to share your model on huggingface.co/models =)




TrainOutput(global_step=800, training_loss=0.3994746758043766, metrics={'train_runtime': 371.6268, 'train_samples_per_second': 17.222, 'train_steps_per_second': 2.153, 'total_flos': 0.0, 'train_loss': 0.3994746758043766, 'epoch': 2.0})

In [145]:
trainer.evaluate()

***** Running Evaluation *****
  Num examples = 800
  Batch size = 16


{'eval_loss': 0.3862309157848358,
 'eval_accuracy': 0.88875,
 'eval_f1': 0.8808567603748326,
 'eval_precision': 0.8844086021505376,
 'eval_recall': 0.8773333333333333,
 'eval_runtime': 14.5483,
 'eval_samples_per_second': 54.989,
 'eval_steps_per_second': 3.437,
 'epoch': 2.0}

In [146]:
trainer.evaluate(eval_dataset=test_dataset, metric_key_prefix="test")

***** Running Evaluation *****
  Num examples = 1000
  Batch size = 16


{'test_loss': 0.3936313986778259,
 'test_accuracy': 0.891,
 'test_f1': 0.8893401015228426,
 'test_precision': 0.8812877263581489,
 'test_recall': 0.8975409836065574,
 'test_runtime': 18.2153,
 'test_samples_per_second': 54.899,
 'test_steps_per_second': 3.459,
 'epoch': 2.0}

# CLS

In [147]:
class SentimentClassifierCLS(nn.Module):

    def __init__(self, n_classes):

    super().__init__()
    self.bert = BertModel.from_pretrained(PRE_TRAINED_MODEL_NAME)
    self.drop = nn.Dropout(p=0.3)
    self.linear = nn.Linear(self.bert.config.hidden_size, self.bert.config.hidden_size)
    self.out = nn.Linear(self.bert.config.hidden_size + 768, n_classes)
    self.n_classes = n_classes

    def forward(self,        
        input_ids=None,
        attention_mask=None,
        token_type_ids=None,
        position_ids=None,
        head_mask=None,
        inputs_embeds=None,
        labels=None,
        output_attentions=None,
        output_hidden_states=None
    ):

    last_hidden_state, pooled_output = self.bert(input_ids=input_ids,
                        attention_mask=attention_mask,
                        return_dict=False)

    cls = last_hidden_state[:,0,:]
    pooled_output = self.linear(self.drop(pooled_output)) 
    stacked_layers = torch.hstack([cls, pooled_output])
    logits = self.out(stacked_layers)

    loss = None
    if labels is not None:
        loss_fct = CrossEntropyLoss()
        loss = loss_fct(logits.view(-1, self.n_classes), labels.view(-1))
    output = (logits,)
    return ((loss,) + output) if loss is not None else output

In [148]:
model = SentimentClassifierCLS(labels_num)
model = model.to(device)

loading configuration file https://huggingface.co/bert-base-uncased/resolve/main/config.json from cache at /root/.cache/huggingface/transformers/3c61d016573b14f7f008c02c4e51a366c67ab274726fe2910691e2a761acf43e.37395cee442ab11005bcd270f3c34464dc1704b715b5d7d52b1a461abe3b9e4e
Model config BertConfig {
  "architectures": [
    "BertForMaskedLM"
  ],
  "attention_probs_dropout_prob": 0.1,
  "classifier_dropout": null,
  "gradient_checkpointing": false,
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.1,
  "hidden_size": 768,
  "initializer_range": 0.02,
  "intermediate_size": 3072,
  "layer_norm_eps": 1e-12,
  "max_position_embeddings": 512,
  "model_type": "bert",
  "num_attention_heads": 12,
  "num_hidden_layers": 12,
  "pad_token_id": 0,
  "position_embedding_type": "absolute",
  "transformers_version": "4.16.2",
  "type_vocab_size": 2,
  "use_cache": true,
  "vocab_size": 30522
}

loading weights file https://huggingface.co/bert-base-uncased/resolve/main/pytorch_model.bin from cache 

In [149]:
trainer = Trainer(
          model=model,
          args=training_args,
          train_dataset=train_dataset,
          eval_dataset=val_dataset,
          compute_metrics=compute_metrics,
)

In [150]:
trainer.train()

/opt/conda/lib/python3.7/site-packages/transformers/optimization.py:309: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use thePyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  FutureWarning,
***** Running training *****
  Num examples = 3200
  Num Epochs = 2
  Instantaneous batch size per device = 8
  Total train batch size (w. parallel, distributed & accumulation) = 8
  Gradient Accumulation steps = 1
  Total optimization steps = 800


Step,Training Loss
10,0.706300
20,0.677100
30,0.668500
40,0.684200
50,0.678200
60,0.673600
70,0.634600
80,0.647000
90,0.577800
100,0.450900


Saving model checkpoint to ./results/checkpoint-500
Trainer.model is not a `PreTrainedModel`, only saving its state dict.


Training completed. Do not forget to share your model on huggingface.co/models =)




TrainOutput(global_step=800, training_loss=0.35496791936457156, metrics={'train_runtime': 371.767, 'train_samples_per_second': 17.215, 'train_steps_per_second': 2.152, 'total_flos': 0.0, 'train_loss': 0.35496791936457156, 'epoch': 2.0})

In [151]:
trainer.evaluate()

***** Running Evaluation *****
  Num examples = 800
  Batch size = 16


{'eval_loss': 0.3989125192165375,
 'eval_accuracy': 0.90625,
 'eval_f1': 0.9009247027741083,
 'eval_precision': 0.8926701570680629,
 'eval_recall': 0.9093333333333333,
 'eval_runtime': 14.5601,
 'eval_samples_per_second': 54.945,
 'eval_steps_per_second': 3.434,
 'epoch': 2.0}

In [152]:
trainer.evaluate(eval_dataset=test_dataset, metric_key_prefix="test")

***** Running Evaluation *****
  Num examples = 1000
  Batch size = 16


{'test_loss': 0.39849212765693665,
 'test_accuracy': 0.9,
 'test_f1': 0.8995983935742973,
 'test_precision': 0.8818897637795275,
 'test_recall': 0.9180327868852459,
 'test_runtime': 18.2028,
 'test_samples_per_second': 54.937,
 'test_steps_per_second': 3.461,
 'epoch': 2.0}

# Model to data

In [160]:
from transformers import BertForSequenceClassification

In [161]:
model = BertForSequenceClassification.from_pretrained(PRE_TRAINED_MODEL_NAME)
model = model.to(device)

loading configuration file https://huggingface.co/bert-base-uncased/resolve/main/config.json from cache at /root/.cache/huggingface/transformers/3c61d016573b14f7f008c02c4e51a366c67ab274726fe2910691e2a761acf43e.37395cee442ab11005bcd270f3c34464dc1704b715b5d7d52b1a461abe3b9e4e
Model config BertConfig {
  "architectures": [
    "BertForMaskedLM"
  ],
  "attention_probs_dropout_prob": 0.1,
  "classifier_dropout": null,
  "gradient_checkpointing": false,
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.1,
  "hidden_size": 768,
  "initializer_range": 0.02,
  "intermediate_size": 3072,
  "layer_norm_eps": 1e-12,
  "max_position_embeddings": 512,
  "model_type": "bert",
  "num_attention_heads": 12,
  "num_hidden_layers": 12,
  "pad_token_id": 0,
  "position_embedding_type": "absolute",
  "transformers_version": "4.16.2",
  "type_vocab_size": 2,
  "use_cache": true,
  "vocab_size": 30522
}

loading weights file https://huggingface.co/bert-base-uncased/resolve/main/pytorch_model.bin from cache 

In [162]:
trainer = Trainer(
          model=model,
          args=training_args,
          train_dataset=train_dataset,
          eval_dataset=val_dataset,
          compute_metrics=compute_metrics,
)

In [163]:
trainer.train()

/opt/conda/lib/python3.7/site-packages/transformers/optimization.py:309: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use thePyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  FutureWarning,
***** Running training *****
  Num examples = 3200
  Num Epochs = 2
  Instantaneous batch size per device = 8
  Total train batch size (w. parallel, distributed & accumulation) = 8
  Gradient Accumulation steps = 1
  Total optimization steps = 800


Step,Training Loss
10,0.712600
20,0.694900
30,0.677300
40,0.679700
50,0.678800
60,0.651700
70,0.676000
80,0.662300
90,0.575900
100,0.522400


Saving model checkpoint to ./results/checkpoint-500
Configuration saved in ./results/checkpoint-500/config.json
Model weights saved in ./results/checkpoint-500/pytorch_model.bin


Training completed. Do not forget to share your model on huggingface.co/models =)




TrainOutput(global_step=800, training_loss=0.37710245288908484, metrics={'train_runtime': 372.5274, 'train_samples_per_second': 17.18, 'train_steps_per_second': 2.147, 'total_flos': 1683910754304000.0, 'train_loss': 0.37710245288908484, 'epoch': 2.0})

In [164]:
trainer.evaluate()

***** Running Evaluation *****
  Num examples = 800
  Batch size = 16


{'eval_loss': 0.3261491358280182,
 'eval_accuracy': 0.89875,
 'eval_f1': 0.8921438082556591,
 'eval_precision': 0.8909574468085106,
 'eval_recall': 0.8933333333333333,
 'eval_runtime': 14.5525,
 'eval_samples_per_second': 54.973,
 'eval_steps_per_second': 3.436,
 'epoch': 2.0}

In [165]:
trainer.evaluate(eval_dataset=test_dataset, metric_key_prefix="test")

***** Running Evaluation *****
  Num examples = 1000
  Batch size = 16


{'test_loss': 0.3445480465888977,
 'test_accuracy': 0.899,
 'test_f1': 0.8999008919722497,
 'test_precision': 0.8714011516314779,
 'test_recall': 0.930327868852459,
 'test_runtime': 18.2345,
 'test_samples_per_second': 54.841,
 'test_steps_per_second': 3.455,
 'epoch': 2.0}

# Google Play reviews

## 1/5 stars

In [167]:
review = "Shows how greed and drugs turn people ugly. Not one character in this movie that makes you feel good. Didn't laugh once, didn't feel good once, didn't learn anything = waste of time (and since it's a long movie, lot's of time wasted). I watched it until the end hoping for something uplifting at the end but it didn't come. Good acting but lot's of foul language. This movie also failed to show where does all the money on Wall Street come from. It missed the opportunity to show how amateur investors lose their hard earned money and the consequences because of the promise of big returns by the Wolf's greedy gang."

In [168]:
encoding = tokenizer.encode_plus(
  review,
  max_length=512,
  add_special_tokens=True, 
  return_token_type_ids=True,
  padding='max_length',
  return_attention_mask=True,
  return_tensors='pt',  
  truncation=True
).to('cuda')

In [169]:
model(
  input_ids=encoding['input_ids'], 
  attention_mask=encoding['attention_mask'])[0].cpu().detach().numpy().argmax()

0

Отзыв был на 1/1, совсем плохой, модель правильно его оценила – как плохой.

## 3/5 stars

In [170]:
review = "It's one of those movies that has you waiting for that moment that never comes. Although it was based on true events, it feels like the film creators didn't offer that 'BOOM!' That brings you and the film to a halt and back to reality. It felt like the movie was advocating the behavior that this lifestyle led M.Belfort to live. Leonardo's acting is extravagant, ruthless and insane at times, which saves the movie's balls at times. Oh, and Jonah Hill was good, too, I suppose. All in all, overrated but OK."

In [171]:
encoding = tokenizer.encode_plus(
  review,
  max_length=512,
  add_special_tokens=True, 
  return_token_type_ids=True,
  padding='max_length',
  return_attention_mask=True,
  return_tensors='pt',  
  truncation=True
).to('cuda')

In [172]:
model(
  input_ids=encoding['input_ids'], 
  attention_mask=encoding['attention_mask'])[0].cpu().detach().numpy().argmax()

0

Отзыв был на 3/5, удовлетворительный. Было интересно посмотреть, что модель про это скажет. Модель оценила его как 0 (отрицательный).

## 5/5 stars

In [173]:
review = "My New Favorite Movie! This movie grabbed my eyes and ears less than 2 mins into it (which is very rare for me), and did not let go until the end. It made me feel everything: Happiness, sadness, anger, jealousy, pity etc. It is a movie about greed, power and pride, and is packed with sex, money, and drugs. And I mean PACKED. So not everyone may enjoy it. But it is a true story about a financially elite man, and his privileged lifestyle. Excellent Movie All Around!"

In [174]:
encoding = tokenizer.encode_plus(
  review,
  max_length=512,
  add_special_tokens=True, 
  return_token_type_ids=True,
  padding='max_length',
  return_attention_mask=True,
  return_tensors='pt',  
  truncation=True
).to('cuda')

In [175]:
model(
  input_ids=encoding['input_ids'], 
  attention_mask=encoding['attention_mask'])[0].cpu().detach().numpy().argmax()

1

Данный отзыв был хороший, аж на 5/5 звезд, модель также оценила его правильно – как хороший.